In [7]:
import pandas as pd

# Remplace 'nom_du_fichier.csv' par le chemin réel de ton fichier CSV
chemin_fichier = 'flickr_data2.csv'

# Lecture du fichier CSV
try:
    data = pd.read_csv(chemin_fichier, sep=',')
    print("Fichier importé avec succès !")
except Exception as e:
    print(f"Une erreur est survenue : {e}")

Fichier importé avec succès !


/var/folders/r2/gpm53c192dsgzq8bxk_h_nph0000gn/T/ipykernel_86286/2091030073.py:8: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(chemin_fichier, sep=',')


In [32]:
print("### Informations générales sur le dataset ###")
print(data.info())

# Nombre total de lignes et colonnes
print("\nNombre total de lignes et colonnes :")
print(f"Lignes : {data.shape[0]}, Colonnes : {data.shape[1]}")

# Statistiques descriptives
print("\n### Statistiques descriptives ###")
#print(data.describe(include='all'))  # Inclut toutes les colonnes

# Comptage des valeurs nulles par colonne
print("\n### Nombre de valeurs nulles par colonne ###")
print(data.isnull().sum())

# Pourcentage de valeurs nulles
print("\n### Pourcentage de valeurs nulles par colonne ###")
print((data.isnull().sum() / data.shape[0]) * 100)

# Détection des doublons
print("\n### Nombre de doublons ###")
print(data.duplicated().sum())

# Aperçu des premières lignes
print("\n### Aperçu des premières lignes ###")
print(data.head())


### Informations générales sur le dataset ###
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420240 entries, 0 to 420239
Data columns (total 19 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   420240 non-null  int64  
 1    user                420240 non-null  object 
 2    lat                 420240 non-null  float64
 3    long                420240 non-null  float64
 4    tags                316730 non-null  object 
 5    title               381911 non-null  object 
 6    date_taken_minute   420239 non-null  float64
 7    date_taken_hour     420240 non-null  int64  
 8    date_taken_day      420240 non-null  int64  
 9    date_taken_month    420240 non-null  int64  
 10   date_taken_year     420240 non-null  int64  
 11   date_upload_minute  420228 non-null  object 
 12   date_upload_hour    420238 non-null  object 
 13   date_upload_day     420238 non-null  float64
 14   date_upload_month   4

In [24]:

try:
    # Suppression des doublons sur toutes les colonnes
    data_sans_doublons = data.drop_duplicates()
    print("Les doublons ont été supprimés du dataset.")

    # Identifier les 3 dernières colonnes
    dernieres_colonnes = data_sans_doublons.columns[-3:]  # Noms des 3 dernières colonnes
    print("### Les 3 dernières colonnes identifiées sont : ###")
    print(dernieres_colonnes)

    # Filtrer les lignes où au moins une des 3 dernières colonnes contient une valeur non nulle
    lignes_problemes = data_sans_doublons[dernieres_colonnes].notnull().any(axis=1)

    # Afficher les lignes problématiques
    print("\n### Lignes contenant des valeurs dans les 3 dernières colonnes : ###")
    lignes_problemes_df = data_sans_doublons[lignes_problemes]
    print(lignes_problemes_df)

except Exception as e:
    print(f"Une erreur est survenue : {e}")


Les doublons ont été supprimés du dataset.
### Les 3 dernières colonnes identifiées sont : ###
Index(['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18'], dtype='object')

### Lignes contenant des valeurs dans les 3 dernières colonnes : ###
                 id             user        lat      long  \
85950    6674970791     29713277@N02  45.753948  4.788145   
98808    7386785280     37290448@N04  45.771315  4.835829   
98816    7387000024     38586649@N00  45.779196  4.853596   
102366   7583546064     37290448@N04  45.772827  4.841065   
102720   7527657538     43706323@N03  45.769222  4.830549   
108687   8030462612     43320999@N06  45.757835  4.830446   
108958   8003036402      9728075@N06  45.723738  4.832203   
116511   8150613256     58993764@N06  45.761180  4.831458   
127024   8442159450     34659993@N04  45.780666  4.872666   
127157   8434131946     71564713@N02  45.748409  4.862918   
129635   8597695772     40633359@N02  45.762501  4.822761   
131976   8442159450     34659993@N

In [26]:
print(lignes_problemes_df.columns)

Index(['id', ' user', ' lat', ' long', ' tags', ' title', ' date_taken_minute',
       ' date_taken_hour', ' date_taken_day', ' date_taken_month',
       ' date_taken_year', ' date_upload_minute', ' date_upload_hour',
       ' date_upload_day', ' date_upload_month', ' date_upload_year',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18'],
      dtype='object')


In [31]:
lignes_problemes_df['title_date_concat'] = (
    lignes_problemes_df[' title'].astype(str) + '_:' + lignes_problemes_df[' date_upload_minute'].astype(str)
)
# Réassigner les colonnes en utilisant .loc
lignes_problemes_df.loc[:, ' date_upload_minute'] = lignes_problemes_df[' date_upload_hour']
lignes_problemes_df.loc[:, ' date_upload_hour'] = lignes_problemes_df[' date_upload_day']
lignes_problemes_df.loc[:, ' date_upload_day'] = lignes_problemes_df[' date_upload_month']
lignes_problemes_df.loc[:, ' date_upload_month'] = lignes_problemes_df[' date_upload_year']
lignes_problemes_df.loc[:, ' date_upload_year'] = lignes_problemes_df['Unnamed: 16']
lignes_problemes_df.loc[:, ' title'] = lignes_problemes_df['title_date_concat']

print(lignes_problemes_df.info())

    # Sauvegarde des lignes problématiques dans un fichier pour analyse
lignes_problemes_df.to_csv("lignes_problemes.csv", index=False)
print("\nLes lignes problématiques ont été sauvegardées dans 'lignes_problemes.csv'.")

<class 'pandas.core.frame.DataFrame'>
Index: 53 entries, 85950 to 384404
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   53 non-null     int64  
 1    user                53 non-null     object 
 2    lat                 53 non-null     float64
 3    long                53 non-null     float64
 4    tags                38 non-null     object 
 5    title               53 non-null     object 
 6    date_taken_minute   52 non-null     float64
 7    date_taken_hour     53 non-null     int64  
 8    date_taken_day      53 non-null     int64  
 9    date_taken_month    53 non-null     int64  
 10   date_taken_year     53 non-null     int64  
 11   date_upload_minute  53 non-null     object 
 12   date_upload_hour    53 non-null     float64
 13   date_upload_day     53 non-null     int64  
 14   date_upload_month   53 non-null     float64
 15   date_upload_year    53 non-null     fl

/var/folders/r2/gpm53c192dsgzq8bxk_h_nph0000gn/T/ipykernel_86286/3888365064.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lignes_problemes_df['title_date_concat'] = (
